# Dicoding Belajar Machine Learning Untuk Pemula Final Project : Image Classification

- **Nama**         : Azarya Yehezkiel Pinondang Sipahutar
- **Email**         : azaryaemc@gmail.com
- **ID Dicoding**   : azarya_yehezkiel

In [38]:
# Import the necessary libraries for file manipulation and visualization
import zipfile, os
import shutil
import numpy as np
from IPython.display import display
from ipywidgets import widgets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import io
from PIL import Image
from IPython.display import display
from ipywidgets import widgets

# Import the necessary libraries for machine learning
from sklearn.model_selection import train_test_split

# Import the necessary libraries for deep learning with TensorFlow
import tensorflow as tf
print(tf.__version__)  # Print the version of TensorFlow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

2.15.0


## Data Preparation

In this section, we load the images from our dataset and perform some preprocessing such as rescaling the images. We also split our dataset into training and validation sets.

In [3]:
# The path to the zip file containing the images is defined
local_zip = 'rockpaperscissors.zip'

# The zip file is opened in read mode
zip_ref = zipfile.ZipFile(local_zip, 'r')

# The contents of the zip file are extracted to the '/tmp' directory
zip_ref.extractall('/tmp')

# The zip file is closed
zip_ref.close()

# The base directory where the images are located is defined
base_dir = '/tmp/rockpaperscissors/rps-cv-images'

# The directories for the 'rock', 'paper', and 'scissors' images are defined
rock_dir = os.path.join(base_dir, 'rock')
paper_dir = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors')

### Split Train(60%) Test (40%) Data

In [4]:
# Define the directories for the training and validation datasets
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# Create these directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Split the data into training and validation sets
# We're using a 60/40 split, so 60% of the images will be used for training, and 40% for validation
# The train_test_split function shuffles the images before splitting them, to ensure a good mix of images in both sets
train_rock_dir, val_rock_dir = train_test_split(os.listdir(rock_dir), test_size = 0.4)
train_paper_dir, val_paper_dir = train_test_split(os.listdir(paper_dir), test_size = 0.4)
train_scissors_dir, val_scissors_dir = train_test_split(os.listdir(scissors_dir), test_size = 0.4)

In [5]:
# Function to move files
def move_files(files, src_dir, dst_dir):
    """
    Move a list of files from a source directory to a destination directory.

    Parameters:
    files (list): A list of filenames to be moved.
    src_dir (str): The directory where the files currently reside.
    dst_dir (str): The directory where the files should be moved to.

    Returns:
    None
    """
    os.makedirs(dst_dir, exist_ok=True)  # Ensure the directory exists
    for file in files:
        shutil.move(os.path.join(src_dir, file), os.path.join(dst_dir, file))

# Move the files for 'rock' category
move_files(train_rock_dir, rock_dir, os.path.join(train_dir, 'rock'))
move_files(val_rock_dir, rock_dir, os.path.join(validation_dir, 'rock'))

# Move the files for 'paper' category
move_files(train_paper_dir, paper_dir, os.path.join(train_dir, 'paper'))
move_files(val_paper_dir, paper_dir, os.path.join(validation_dir, 'paper'))

# Move the files for 'scissors' category
move_files(train_scissors_dir, scissors_dir, os.path.join(train_dir, 'scissors'))
move_files(val_scissors_dir, scissors_dir, os.path.join(validation_dir, 'scissors'))

In [6]:
# show train directory
os.listdir(train_dir)

['paper', 'rock', 'scissors']

In [7]:
# show validation directory
os.listdir(validation_dir)

['paper', 'rock', 'scissors']

In [8]:
# Create an instance of ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0,1]
    rotation_range=20,  
    horizontal_flip=True,  
    shear_range = 0.2,  
    fill_mode = 'nearest',  
)

# Load images from the disk, applies data augmentation, and resizes the images
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150),
    batch_size=5,
    class_mode='categorical'  # 'categorical' for multi-class labels
)

Found 2181 images belonging to 3 classes.


In [9]:
# Create an instance of ImageDataGenerator for validation data
test_datagen = ImageDataGenerator(
                    rescale=1./255 # Normalize pixel values  
)

# Load images from the disk, rescale them, and resize tahe images
validation_generator = test_datagen.flow_from_directory(
    validation_dir,  
    target_size=(150, 150),
    batch_size=5,
    class_mode='categorical'  
)

Found 2089 images belonging to 3 classes.


In [10]:
# Create a dictionary to map the class indices to their respective labels
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
# Print each class index and its corresponding label
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for classes present in the training and validation datasets

0 : paper
1 : rock
2 : scissors


## Model Building

Here we build our image classification model. We're using a Convolutional Neural Network (CNN) which is commonly used in image classification tasks.

In [11]:
# Define the model architecture
model = tf.keras.models.Sequential([
    # First convolution layer, 32 filters of size 3x3, activation function is ReLU
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    # First max pooling layer with pool size 2x2
    tf.keras.layers.MaxPooling2D(2,2),

    # Second convolution layer, 64 filters of size 3x3, activation function is ReLU
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    # Second max pooling layer with pool size 2x2
    tf.keras.layers.MaxPooling2D(2,2),

    # Third convolution layer, 128 filters of size 3x3, activation function is ReLU
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    # Third max pooling layer with pool size 2x2
    tf.keras.layers.MaxPooling2D(2,2),

    # Fourth convolution layer, 256 filters of size 3x3, activation function is ReLU
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    # Fourth max pooling layer with pool size 2x2
    tf.keras.layers.MaxPooling2D(2,2),

    # Flatten layer to convert the 3D feature maps to 1D feature vectors
    tf.keras.layers.Flatten(),

    # Fully connected layer with 512 neurons, activation function is ReLU
    tf.keras.layers.Dense(512, activation='relu'),
    # Fully connected layer with 256 neurons, activation function is ReLU
    tf.keras.layers.Dense(256, activation='relu'),

    # Output layer with 3 neurons (for 3 classes), activation function is softmax
    tf.keras.layers.Dense(3, activation='softmax')
])

In [12]:
# summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [13]:
# Compile the model
model.compile(
    # Use RMSprop optimizer
    optimizer=tf.optimizers.RMSprop(),
    # Use Kullback-Leibler divergence as the loss function
    loss='kullback_leibler_divergence',
    # Track accuracy as a metric during training
    metrics=['accuracy']
)

## Model Training

Now that our model is built, we can train it using our training data. We also validate our model using the validation data.

In [14]:
class CallbackEpoch(tf.keras.callbacks.Callback):
    """
    This class inherits from the `tf.keras.callbacks.Callback` class and overrides the `on_epoch_end` method.
    It's used to stop the training process once a certain accuracy threshold is reached.
    """
    def on_epoch_end(self, epoch, logs=None):
        # If accuracy > 98%, stop training
        if logs.get('accuracy') > 0.98 and logs.get('val_accuracy') > 0.98:
            print("\nReached 98% accuracy so cancelling training!")
            self.model.stop_training = True

# Instantiate the custom callback
callback_epoch = CallbackEpoch()

# Train Model 
model.fit(train_generator, 
          epochs=10, 
          validation_data=validation_generator, 
          verbose=2, 
          callbacks=[callback_epoch])

Epoch 1/10


437/437 - 29s - loss: 0.6571 - accuracy: 0.7093 - val_loss: 0.1817 - val_accuracy: 0.9449 - 29s/epoch - 65ms/step
Epoch 2/10
437/437 - 28s - loss: 0.2715 - accuracy: 0.9156 - val_loss: 0.1176 - val_accuracy: 0.9674 - 28s/epoch - 64ms/step
Epoch 3/10
437/437 - 31s - loss: 0.1877 - accuracy: 0.9390 - val_loss: 0.0891 - val_accuracy: 0.9732 - 31s/epoch - 71ms/step
Epoch 4/10
437/437 - 29s - loss: 0.1509 - accuracy: 0.9509 - val_loss: 0.2253 - val_accuracy: 0.9167 - 29s/epoch - 66ms/step
Epoch 5/10
437/437 - 30s - loss: 0.1279 - accuracy: 0.9647 - val_loss: 0.1006 - val_accuracy: 0.9775 - 30s/epoch - 69ms/step
Epoch 6/10
437/437 - 29s - loss: 0.1628 - accuracy: 0.9619 - val_loss: 0.0556 - val_accuracy: 0.9885 - 29s/epoch - 67ms/step
Epoch 7/10
437/437 - 29s - loss: 0.1090 - accuracy: 0.9702 - val_loss: 0.0410 - val_accuracy: 0.9904 - 29s/epoch - 66ms/step
Epoch 8/10

Reached 98% accuracy so cancelling training!
437/437 - 30s - loss: 0.0790 - accuracy: 0.9826 - val_loss: 0.0279

## Model Implementation to new pictures

In [37]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt


# Create a FileUpload widget
uploader = widgets.FileUpload(multiple=False)
display(uploader)

def on_upload_change(change):
    # Get the uploaded file
    uploaded = change['new']
    for fn, file_info in uploaded.items():
        with io.BytesIO(file_info['content']) as f:
            img = Image.open(f).resize((150, 150))
        imgplot = plt.imshow(img)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])

        print("Image preprocessed")  # Debug print

        # Make a prediction
        classes = model.predict(images, batch_size=10)
        print("Prediction made:", classes)  # Debug print

        print(fn)
        if classes[0,0] != 0:
            print('paper')
        elif classes[0,1] != 0:
            print('rock')
        else:
            print('scissors')

# Attach the function to the uploader
uploader.observe(on_upload_change, names='_counter')

FileUpload(value=(), description='Upload')